In [1]:
!pip install pytesseract transformers[torch] datasets rouge-score nltk py7zr --upgrade #tensorboard
!pip install accelerate -U
!pip install bert_score
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.5 

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

dataset_id = "knkarthick/dialogsum"
dataset = load_dataset(dataset_id)
model_id="google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [3]:
dataset["train"][0]["topic"]

'get a check-up'

In [5]:
import json
from datasets import concatenate_datasets
from datasets.dataset_dict import DatasetDict
ds = DatasetDict()

# preprocess
with open("train_sick.json", "r", encoding="utf8") as f:
    train_sick = json.load(f)

def update_dataset(example, index):
    example["dialogue"] = f"Summarize the following dialogue with helping information after each utterance (in the next newline between the special separator) and towards the topic of {dataset['train'][index]['topic']}.\n" + train_sick[index]
    return example

ds["train"] = dataset["train"].map(update_dataset, with_indices=True)

with open("test_sick.json", "r", encoding="utf8") as f:
    test_sick = json.load(f)

def update_dataset(example, index):
    example["dialogue"] = f"Summarize the following dialogue with helping information after each utterance (in the next newline between the special separator) and towards the topic of {dataset['test'][index]['topic']}.\n" + test_sick[index]
    return example

ds["test"] = dataset["test"].map(update_dataset, with_indices=True)


tokenized_inputs = concatenate_datasets([ds["train"], ds["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])

tokenized_targets = concatenate_datasets([ds["train"], ds["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])

def preprocess_function(sample,padding="max_length"):

    # the instructions can be rewritten based on need

    # NOTE I tried the <I></I> and <sep></sep>, not much difference

    # and towards the topic of {sample['topic']}.

    # the raw instruction
    # inputs = ["Summarize the following dialogue. \n" + item for item in sample["dialogue"]]

    # The topic and full instruction (added previously)
    inputs = [item for item in sample["dialogue"]]

    # The full instruction (topic added previously) old version
    # inputs = [f"Summarize the following dialogue with helping information after each utterance (between the special separator) and" + item for item in sample["dialogue"]]

    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# the problem is here
tokenized_dataset = ds.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
# tokenized_dataset["train"] = updated_train.map(preprocess_function, with_indices=True, batched=True,remove_columns=["dialogue", "summary", "id"])
# tokenized_dataset["test"] = updated_test.map(preprocess_function, with_indices=True, batched=True,remove_columns=["dialogue", "summary", "id"])


Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/13960 [00:00<?, ? examples/s]

Map:   0%|          | 0/13960 [00:00<?, ? examples/s]

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
ds["test"]["dialogue"][0]

In [ ]:
tokenized_dataset["test"]["input_ids"][0] == tokenized_dataset["test"]["input_ids"][1]

In [6]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [8]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    fp16=False,
    learning_rate=5e-5,
    num_train_epochs=2,
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
# trainer.evaluate()

In [ ]:
# {'eval_loss': 1.065560221672058,
#  'eval_rouge1': 41.055,
#  'eval_rouge2': 18.0998,
#  'eval_rougeL': 35.6308,
#  'eval_rougeLsum': 37.4082,
#  'eval_gen_len': 18.845333333333333,
#  'eval_runtime': 373.4175,
#  'eval_samples_per_second': 4.017,
#  'eval_steps_per_second': 1.004,
#  'epoch': 2.0}

In [ ]:
def my_tokenize(examples):
    return tokenizer(examples['dialogue'], padding=True, truncation=True, max_length=512)
td = ds["test"].map(my_tokenize, batched=True)

In [ ]:
ps = trainer.predict(td, max_length = 100)




In [ ]:
import numpy as np
predictions = np.where(ps.predictions != -100, ps.predictions, tokenizer.pad_token_id)
predictions = [tokenizer.decode(ids, skip_special_tokens=True, max_length = 100) for ids in predictions]
references = [s["summary"] for s in dataset["test"]]

In [ ]:
print(predictions[0])
print(predictions[1])
print(predictions[2])
print(predictions[10])
print(predictions[11])
print(predictions[12])

In [ ]:
from rouge_score import rouge_scorer, scoring

def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()

    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        aggregator.add_scores(scores)

    # 计算最终的平均分数
    result = aggregator.aggregate()

    # 转换为易于阅读的格式
    final_scores = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return final_scores

rouge_scores = compute_rouge(predictions, references)
print(rouge_scores)


In [ ]:
# from evaluate import load
# bertscore = load("bertscore")

# results = bertscore.compute(predictions=predictions, references=references, lang="en")

In [ ]:
# predictions
with open("flan_t5_base_dialogsum_full.txt", 'w', encoding='utf-8') as f:
    for string in predictions:
        f.write(string + '\n')

from google.colab import files
files.download('flan_t5_base_dialogsum_full.txt')